<a href="https://colab.research.google.com/github/FabioSicca/Organizacion-de-datos/blob/main/Pandas4_TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

42) Si la experiencia de un usuario es la cantidad de logs en los que participó, ¿cuál es la tasa de contenidos cuya última revisión no tiene comentario en función de la experiencia de su revisor? (⭐⭐⭐)

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
logs = pd.read_csv('/content/drive/MyDrive/Wikipedia dataset/logs.csv')
contents = pd.read_csv('/content/drive/MyDrive/Wikipedia dataset/contents.csv')

<ipython-input-2-447a0121edcc>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  logs = pd.read_csv('/content/drive/MyDrive/Wikipedia dataset/logs.csv')


Limpio el df 'logs' y elimino los NaN de 'contributor_id' ya que no me sirven

In [ ]:
logs_filtered = logs[["item_id","contributor_username", "contributor_id", "title"]]
logs_filtered = logs_filtered.dropna(subset=["contributor_id"])

#logs_filtered.info()
#logs_filtered
#contents

Limpio el df 'contents' para trabajar con lo que me interesa y elimino las filas que no tienen un revisor_id

In [ ]:
contents_filtered = contents[["title", "id", "revision_id", "revisor_username", "revisor_id", "revisor_comment"]]
contents_filtered = contents_filtered.dropna(subset=['revisor_id'])

contents_filtered.head(5)

,title,id,revision_id,revisor_username,revisor_id,revisor_comment
0,Wikipedia:Artículos solicitados,5,132533307,Metrónomo,1708233.0,Así esta mejor
2,Argentina,10,137966826,Traitrich,4642283.0,Reemplazos con [[Usuario:Benjavalero/Replacer|...
3,Geografía de Andorra,15,137986788,Lojwe,5338508.0,Reemplazos con [[Usuario:Benjavalero/Replacer|...
4,Demografía de Andorra,17,130869554,Jialxv,3615366.0,Correcciones ortográficas con [[Usuario:Benjav...
5,Comunicaciones de Andorra,18,132654704,PatriHorrillo,689866.0,Correcciones ortográficas con [[Usuario:Benjav...


Devuelve una serie con la cantidad de logs segun el contributor_id (es decir, su experiencia)

In [ ]:
user_experience = logs.groupby("contributor_id").size()
user_experience

contributor_id
5.0          8
21.0         1
30.0         1
38.0         9
206.0        1
            ..
6335012.0    5
6335013.0    1
6335017.0    1
6335019.0    1
6335020.0    1
Length: 6188938, dtype: int64

Creo una columna nueva llamada 'revisor_xp' y en base a la serie que teniamos con la experiencia de cada revisor, le agrego la experiencia del revisor a la fila de contents

In [ ]:
contents_filtered['revisor_xp'] = contents_filtered['revisor_id'].map(user_experience)
contents_filtered.head(3)

,title,id,revision_id,revisor_username,revisor_id,revisor_comment,revisor_xp
0,Wikipedia:Artículos solicitados,5,132533307,Metrónomo,1708233.0,Así esta mejor,11256.0
2,Argentina,10,137966826,Traitrich,4642283.0,Reemplazos con [[Usuario:Benjavalero/Replacer|...,236.0
3,Geografía de Andorra,15,137986788,Lojwe,5338508.0,Reemplazos con [[Usuario:Benjavalero/Replacer|...,1049.0


Dado que en contents hay paginas que tienen un 'revisor_id' que no aparece en el df de logs la experiencia de éstos aparece como NaN, por lo que decido eliminarlos porque no tiene sentido trabajar con ellos.

In [ ]:
contents_filtered = contents_filtered.dropna(subset=["revisor_xp"])
contents_filtered.head(3)

,title,id,revision_id,revisor_username,revisor_id,revisor_comment,revisor_xp
0,Wikipedia:Artículos solicitados,5,132533307,Metrónomo,1708233.0,Así esta mejor,11256.0
2,Argentina,10,137966826,Traitrich,4642283.0,Reemplazos con [[Usuario:Benjavalero/Replacer|...,236.0
3,Geografía de Andorra,15,137986788,Lojwe,5338508.0,Reemplazos con [[Usuario:Benjavalero/Replacer|...,1049.0


Ahora deberia agrupar por revisor_xp y contar en cada grupo cuantos tienen un NaN en el comentario sobre el total del grupo.

Agrupo la cantidad de contenido por su revisor_xp (CASOS TOTALES)

In [ ]:
contents_filtered_by_xp = contents_filtered.groupby(["revisor_xp"]).size()
contents_filtered_by_xp

revisor_xp
1.0         200154
2.0          45460
3.0          11877
4.0          12743
5.0          46097
             ...  
112565.0      1935
163772.0     61756
193345.0      7218
194876.0      4197
379263.0         1
Length: 1117, dtype: int64

Creo un df que contiene solo aquellos contenidos que no tienen comentarios en su ultima revision y agrupo por experiencia y obtengo la cantidad de contenido que no fue comentado en su revision final (CASOS FAVORABLES, es decir aquellos que cumplen que no tienen comentario)

In [ ]:
contents_without_comment = contents_filtered[contents_filtered["revisor_comment"].isna()]
contents_without_comment_by_xp = contents_without_comment.groupby(["revisor_xp"]).size()
contents_without_comment_by_xp

revisor_xp
1.0         14972
2.0          5187
3.0          3960
4.0          2723
5.0          1684
            ...  
96095.0     29160
101765.0        4
112565.0      364
193345.0      691
194876.0      530
Length: 1046, dtype: int64

Finalmente obtengo la tasa de contenidos cuya última revisión no tiene comentarios en función de la experiencia de su revisor

In [ ]:
tasa_final = contents_without_comment_by_xp / contents_filtered_by_xp

#Cuando un nivel de experiencia deja siempre comentarios, no aparecerá en 'contents_without_comment_by_xp'
#por lo que al hacer la división esto resulta en un NaN, entonces corregimos esto diciendo que su tasa
#de revision sin comentarios es igual a 0.
tasa_final = tasa_final.fillna(0)

tasa_final

revisor_xp
1.0         0.074802
2.0         0.114100
3.0         0.333418
4.0         0.213686
5.0         0.036532
              ...   
112565.0    0.188114
163772.0    0.000000
193345.0    0.095733
194876.0    0.126281
379263.0    0.000000
Length: 1117, dtype: float64

Se puede ver que la tasa de contenido cuya última revisión no tiene comentario de revisores con experiencia igual a 1 es: 0.074802.

Así mismo la última revisión de contenido hecha por revisores con experiencia igual a 2 tiene una tasa de "sin comentarios" de 0.114100, y así sucesivamente.

In [ ]:
tasa_final_porcentaje = tasa_final * 100
tasa_final_porcentaje

revisor_xp
1.0          7.480240
2.0         11.410031
3.0         33.341753
4.0         21.368595
5.0          3.653166
              ...    
112565.0    18.811370
163772.0     0.000000
193345.0     9.573289
194876.0    12.628068
379263.0     0.000000
Length: 1117, dtype: float64

Un 7,48% de los contenidos revisados en su última versión por revisores de experiencia 1 no tienen un comentario